<a href="https://colab.research.google.com/github/ZyCromerZ/sd-webui-colab/blob/main/Copy_of_Model_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Model DL (NoCrypt#6416)**


Can be used to download models and distribute them to Hugging Face and Google Drive (aka. Mirror)<br>
Can also be used to covert ckpt to safetensors<br>
<fieldset>
    <legend>
    <h2><b>Why?</b></h2>
    </legend>

<i>Because not everyone have access to a fast internet (incl. myself), therefore this colab can help them download all models they want and distribute them to Hugging Face and Google Drive without needing them to download to their computer, **which will take them ages to do so**.</i>
</fieldset>

In [ ]:
#@title # 👇 **Download model (can be run multiple times)**
%cd /content
import os
link = "" #@param{type:"string"}
from_direct_link = True #@param{type:"boolean"}
from_gdrive = False #@param{type:"boolean"}
from_torrent = False #@param{type:"boolean"}

!mkdir -p /content/models
%cd /content/models

if from_gdrive:
  !pip install -q -U gdown
  !gdown -c --fuzzy "{link}"

if from_torrent or from_direct_link:
  !apt install -y -qq aria2

if from_torrent:
  !aria2c --summary-interval=10 --seed-ratio=0.1 --allow-overwrite=true "{link}"

if from_direct_link:
  !aria2c  --summary-interval=10 -c -x 10 -k 1M -s 10 "{link}"
  # user_header = f"\"Authorization: Bearer [HUGGING FACE TOKEN HERE]\""
  # !aria2c  --summary-interval=10 -c --header={user_header} -x 10 -k 1M -s 10 "{link}"

# Uncomment if necessary

# Move any file inside a folder if there's any
# !nohup mv **/*.zip /content/models
# !nohup mv **/*.ckpt /content/models
# !nohup mv **/*.pt /content/models

# Unzip them if there is any (modify if needed)
# !nohup unzip *.zip *.rar
# !nohup tar *.tar *.tar
# !rm -rf nohup.out

In [ ]:
# !rm -rf /content/diffusers
# !mkdir -p /content/models
link = "https://huggingface.co/eimiss/EimisAnimeDiffusion_1.0v" #@param{type:"string"}
branch = "main" #@param{type:"string"}
model_name = "EimisAnimeDiffusion_1.0v-fp32" #@param{type:"string"}
# !sudo apt install git-lfs
# !git lfs clone {link} -b {branch} /content/diffusers
# !wget https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_diffusers_to_original_stable_diffusion.py -O /content/convert_diffusers_to_original_stable_diffusion.py
# !pip install "git+https://github.com/huggingface/diffusers@main#egg=diffusers[torch]"
!python3 /content/convert_diffusers_to_original_stable_diffusion.py --model_path "/content/diffusers" --checkpoint_path "/content/models/{model_name}.ckpt"

In [ ]:
#@title # 👇 **Convert to safetensor?** (in progress)
%cd /content/models
import os
!pip install -q torch safetensors
import torch
from safetensors.torch import save_file
def get_file_list(path):
  res = []
  for (dir_path, dir_names, file_names) in os.walk(path):
      res.extend(file_names)
  return res
#@markdown If NAI based, use pop dict. 
pop_dict = False #@param {"type":"boolean"}
delete_original_ckpt = False #@param {"type":"boolean"}

for the_model in get_file_list("/content/models"):
  if os.path.splitext(os.path.basename(the_model))[1] == ".ckpt":
    weights = torch.load(the_model)["state_dict"]
    if pop_dict:
      weights.pop("state_dict")
    print("Converting",the_model,", please wait...")
    save_file(weights, os.path.splitext(os.path.basename(the_model))[0]+".safetensors")
    print("Success.")
    if delete_original_ckpt:
      if os.path.exists(os.path.join("/content/models",the_model)):
        !rm -f /content/models/{the_model}

    weights = "" # Clear ram
    


In [ ]:
#@title # 👇 **Convert from diffusers**
# !rm -rf /content/diffusers
!mkdir -p /content/models
link = "https://huggingface.co/eimiss/EimisAnimeDiffusion_1.0v" #@param{type:"string"}
branch = "main" #@param{type:"string"}
model_name = "EimisAnimeDiffusion_1.0v-fp32" #@param{type:"string"}
!sudo apt install git-lfs
!git clone {link} -b {branch} /content/diffusers
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_diffusers_to_original_stable_diffusion.py -O /content/convert_diffusers_to_original_stable_diffusion.py
!pip install "git+https://github.com/huggingface/diffusers@main#egg=diffusers[torch]"
!python3 /content/convert_diffusers_to_original_stable_diffusion.py --model_path "/content/diffusers" --checkpoint_path "/content/models/{model_name}.ckpt"

In [ ]:
#@title # 👇 **Upload them to Google Drive**
!rm -f *.zip*
from google.colab import drive
import os
if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
%cd /content/models
output_path = "ai_models/" #@param {type:"string"}
!rsync -a -f"- */" -f"+ *" --progress . "/content/gdrive/MyDrive/"{output_path}

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import clear_output

!pip install -q huggingface_hub
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

models_path= "/content/models"
upload_path= '/content/upload_models'

#@title # **👇 Upload them to HuggingFace 🚀🤗**
#@markdown ## **How to use this?**<br>
#@markdown 1. Grab your huggingface **write token** from [here](https://huggingface.co/settings/tokens)
#@markdown 2. Paste the token, then write your repo name
#@markdown 3. Run the cell
#@markdown 4. Select model you want to upload (use `ctrl/shift` for multiple selection)
#@markdown 5. Click on upload button

api = HfApi()
write_token = ""  #@param{type:"string"}
if not os.path.exists('/root/.huggingface/token'):
  !mkdir /root/.huggingface/
  !touch /root/.huggingface/token
f = open("/root/.huggingface/token", "w+")
f.write(write_token)
f.close()

user = api.whoami(write_token)
repo = "EimisAnimeDiffusion_1.0v" #@param{type:"string"}
username_repo = user['name']+"/"+repo.strip()
validate_repo_id(username_repo)

try:
  api.create_repo(repo_id=username_repo)
  print("Repo didn't exists, creating repo")
  print("Repo",username_repo,"created!")
except HfHubHTTPError as e:
  print("Repo exists, skipping create repo")
  
#@markdown ####<br> **⚠️ Rerun this cell if your colab crashed. Because sometimes it did 🫤**<br>
#@markdown *If error, maybe you have "read token" instead of "write token"*
def get_file_list(path):
  res = []
  for (dir_path, dir_names, file_names) in os.walk(path):
      res.extend(file_names)
  return res
  
selected = widgets.SelectMultiple(
    options=get_file_list(models_path),
    rows=10,
    disabled=False,
)

button = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='success',
    tooltip='Upload to huggingface',
)


out = widgets.Output()

def upload_it(b):
    with out:
        if selected.value is not None:
            clear_output()
            !mkdir -p {upload_path}

            #hard link each file
            for selected_model in selected.value:
              if not os.path.exists(os.path.join(upload_path,selected_model)):
                os.link(os.path.join(models_path,selected_model),os.path.join(upload_path,selected_model)) #hardlinking to save colab's space
            
            #delete .ipynb_checkpoint
            if os.path.exists(os.path.join(upload_path,".ipynb_checkpoints")):
              !rm {upload_path}/.ipynb_checkpoints
            print("Selected:", ", ".join(selected.value))
            print("Uploading to https://huggingface.co/"+username_repo)
            print("Please wait...")

            #upload
            api.upload_folder(
                folder_path=upload_path,
                repo_id=username_repo,
                commit_message="Upload with 🚀🤗 NoCrypt's ModelDL"
            )
            
            print("Done!")
            #delete hardlink
            !rm -rf {upload_path}/*

        else:
            print("Nothing is selected")
            b.close()
            selected.close()

button.on_click(upload_it)
print("Upload target: https://huggingface.co/"+username_repo)
print("👇 Select models you want to upload (use ctrl/shift for multiple selection) ")
display(selected,button,out)


# OLD CODE #
# # 👇 **Upload them to Hugging Face**
# If error, maybe you have "read token" instead of "write token", or repo is already exists/not exists
# <br> [GET WRITE TOKEN HERE](https://huggingface.co/settings/tokens)
# !rm -f *.zip*
# import os
# !nohup pip install huggingface_hub > /dev/null
# from huggingface_hub import HfApi
# from huggingface_hub.utils import validate_repo_id
# if os.path.exists("/content/models/.ipynb_checkpoints"): 
#   !rm -rf /content/models/.ipynb_checkpoints
# api = HfApi()
# write_token = ""  #@param{type:"string"}
# if not os.path.exists('/root/.huggingface/token'):
#   !mkdir /root/.huggingface/
#   !touch /root/.huggingface/token
# f = open("/root/.huggingface/token", "w+")
# f.write(write_token)
# f.close()

# user = api.whoami(write_token)
# repo = "models" #@param{type:"string"}
# username_repo = user['name']+"/"+repo
# validate_repo_id(username_repo)
# is_new = "new" #@param["new","existing"]
# if is_new == "new":
#   api.create_repo(repo_id=username_repo)
  
# api.upload_folder(
#     folder_path="/content/models",
#     repo_id=username_repo,
# )




In [ ]:
#@title # 👇 **Remove all files (to download another model)**
!rm -rf /content/models/*
%cd /content/models